In [1]:
from utils import *

from tqdm import tqdm
from pprint import pprint
import json
import _pickle as pkl
from time import time
from random import shuffle, seed

In [2]:
glove = setup_glove()
print(glove.vectors.size())
VOCAB_SIZE = glove.vectors.size()[0]
with open('../data/data.json', 'r') as f:
    data = json.load(f)

torch.Size([400004, 50])


In [9]:
idx = 5
example_X = (data['X_train'][idx])
example_y = (data['y_train'][idx])
print("ID:", example_X[0])
print("Context:", example_X[1])
print("Question:", example_X[2])
print("Answer Span:", example_y)
print("Answer:", example_X[3])

_, p, X, y = make_data([example_X], [example_y], 1, glove)
print(y)
print(get_answer_span(y[0], p[0], example_X[1]))
print("===========")

seed(1)
zipped_data = list(zip(data['X_train'], data['y_train']))
# shuffle(zipped_data)
data['X_train'], data['y_train'] = list(zip(*zipped_data))

num_ex_train = 512
num_ex_val = 32

idxs_train, padlens_train, X_train, y_train = make_data(data['X_train'], data['y_train'], num_ex_train, glove)
idxs_val, padlens_val, X_val, y_val = make_data(data['X_val'], data['y_val'], num_ex_val, glove)
print(len(X_train), len(y_train), len(X_val), len(y_val))

ID: 56ce602faab44d1400b88713
Context: A solar chimney (or thermal chimney, in this context) is a passive solar ventilation system composed of a vertical shaft connecting the interior and exterior of a building. As the chimney warms, the air inside is heated causing an updraft that pulls air through the building. Performance can be improved by using glazing and thermal mass materials in a way that mimics greenhouses.
Question: What kind of system is a solar chimney?
Answer Span: [59, 84]
Answer: passive solar ventilation
[(547, 549)]
passive solar ventilation
512 512 32 32


In [10]:
from models import *

conf = {"vocab": glove.vectors,
        "learning_rate": 0.01,
        "epochs": 5,
        "hidden_size": 32,
        "batch_size": 32,
        "opt": "Adamax",
        "n_layers": 1}

model = ModelV2(conf)
print(model)
model_name = "%s_D%s_B%s_E%s_H%s_LR%s_O%s"%(type(model).__name__, num_ex_train, model.batch_size, model.epochs, model.hidden_size, model.lr, conf["opt"])
print(model_name)

ModelV2(
  (encoder_c): Embedding(400004, 50)
  (encoder_q): Embedding(400004, 50)
  (gru_c): GRU(50, 32)
  (gru_q): GRU(50, 32)
  (lin_q): Linear(in_features=32, out_features=32, bias=True)
  (gru_coatt): GRU(96, 128, bidirectional=True)
  (gru_bmod): GRU(256, 32)
  (ans_ptr_1): Linear(in_features=256, out_features=32, bias=True)
  (ans_ptr_2): Linear(in_features=32, out_features=32, bias=True)
  (ans_ptr_3): Linear(in_features=32, out_features=1, bias=True)
  (decoder_start): Linear(in_features=128, out_features=600, bias=True)
  (decoder_end): Linear(in_features=128, out_features=600, bias=True)
)
ModelV2_D512_B32_E5_H32_LR0.01_OAdamax


In [ ]:
tic = time()
v_preds, losses, vlosses = model.fit((X_train, y_train), (X_val, y_val))
toc = time()
print("took", toc-tic, "seconds")
torch.save(model, '../evaluation/models/%s'%model_name)
import matplotlib.pyplot as plt
plt.plot(list(range(len(losses))), losses, label='train')
plt.plot(list(range(len(vlosses))), vlosses, label='val')
plt.legend()
plt.show()

In [11]:
# model = torch.load('../evaluation/models/%s'%model_name)
model = torch.load('../evaluation/models/ModelV2_D512_B32_E5_H32_LR0.01_OAdamax')
print(model)

ModelV2(
  (encoder_c): Embedding(400004, 50)
  (encoder_q): Embedding(400004, 50)
  (gru_c): GRU(50, 32)
  (gru_q): GRU(50, 32)
  (lin_q): Linear(in_features=32, out_features=32, bias=True)
  (gru_coatt): GRU(96, 128, bidirectional=True)
  (gru_bmod): GRU(256, 32)
  (ans_ptr_1): Linear(in_features=256, out_features=32, bias=True)
  (ans_ptr_2): Linear(in_features=32, out_features=32, bias=True)
  (ans_ptr_3): Linear(in_features=32, out_features=1, bias=True)
  (decoder_start): Linear(in_features=128, out_features=600, bias=True)
  (decoder_end): Linear(in_features=128, out_features=600, bias=True)
)


In [15]:
num_test = 128
idxs_test, padlens_test, X_test, y_test = make_data(data['X_test'], data['y_test'], num_test, glove)

dev_results = {}

def switch_idxs(pred):
    if pred[0]>pred[1]:
        pred[0], pred[1] = pred[1], pred[0]
    return pred

bs = 32
print("Test data size:", num_test)
for bindex,  i in tqdm(enumerate(range(0, len(y_test)-bs+1, bs))):
    print("batch:", bindex)
    # model.init_params(bs)
    Xb = torch.LongTensor(X_test[i:i+bs])
    yb = var(torch.LongTensor(y_test[i:i+bs]))
    
    pred = model.predict(Xb).data.tolist()
    pred = list(map(switch_idxs, pred))
    qids = [data['X_test'][j][0] for j in idxs_test[i:i+bs]]
    test_paras = [data['X_test'][j][1] for j in idxs_test[i:i+bs]]
    pads = padlens_test[i:i+bs]
    answers = list(map(get_answer_span, pred, pads, test_paras))
    batch_results = list(zip(qids, answers))
    # print(len(dict(batch_results)))
    dev_results.update(dict(batch_results))

0it [00:00, ?it/s]

Test data size: 128
batch: 0


1it [00:00,  1.31it/s]

batch: 1


2it [00:01,  1.19it/s]

batch: 2


3it [00:02,  1.20it/s]

batch: 3


4it [00:03,  1.20it/s]


In [16]:
num_processed = len(dev_results)
print("processed:", num_processed)
dev_results['version'] = '1.1'
fname = 'run_%s_test%s.json'%(model_name, num_processed)
with open('../data/%s'%fname, 'w') as f:
    json.dump(dev_results, f)

processed: 43
